In [3]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from bs4 import BeautifulSoup
import requests

In [4]:
# scrape list of NBA teams from Wikipedia

teams = []

r = requests.get("https://en.wikipedia.org/wiki/National_Basketball_Association")
text = r.text
soup = BeautifulSoup(text, 'html.parser')
rows = soup.find("table", attrs={"class": "navbox wikitable"}).find_all("td")
for row in rows:
    elements = row.find_all("b")
    for el in elements:
        teams.append(el.text)
        
print len(teams)

30


In [5]:
# download ticket data into a dataframe

count = 0
for team in teams:
    team_lower = team.lower()
    team_final = team_lower.replace(" ", "+")
    r = requests.get('http://api.seatgeek.com/2/events?q=' + team_final + '&per_page=1000')
    js = r.json()
    events = js["events"]

    df = pd.DataFrame()


    if count == 0:
        for event in events:
            keys =  event.keys()
            break

    for key in keys:
        lst = []
        listing_count = []
        avg_price = []
        lowest_price_good_deal = []
        lowest_price = []
        highest_price = []
        address = []
        city = []
        country = []
        display_location = []
        extended_address = []
        venueid = []
        location = []
        venuename = []
        postal_code = []
        venuescore = []
        venueslug = []
        state = []
        timezone = []
        url = []
    
    
    
    
    
        for event in events:
            if key == "stats":
                listing_count.append(event[key]["listing_count"])
                avg_price.append(event[key]["average_price"])
                lowest_price_good_deal.append(event[key]["lowest_price_good_deals"])
                lowest_price.append(event[key]["lowest_price"])
                highest_price.append(event[key]["highest_price"])
            elif key == "venue":
                address.append(event[key]["address"])
                city.append(event[key]["city"])
                country.append(event[key]["country"])
                display_location.append(event[key]["display_location"])
                extended_address.append(event[key]["extended_address"])
                venueid.append(event[key]["id"])
                location.append(event[key]["location"])
                venuename.append(event[key]["name"])
                postal_code.append(event[key]["postal_code"])
                venuescore.append(event[key]["score"])
                venueslug.append(event[key]["slug"])
                state.append(event[key]["state"])
                timezone.append(event[key]["timezone"])
                url.append(event[key]["url"])
            else:
                lst.append(event[key])
        if key == "stats":
            df["listing_count"] = listing_count
            df["avg_price"] = avg_price
            df["lowest_price_good_deal"] = lowest_price_good_deal
            df["lowest_price"] = lowest_price
            df["highest_price"] = highest_price
        elif key == "venue":
            df["address"] = address
            df["city"] = city
            df["country"] = country
            df["display_location"] = display_location
            df["extended_address"] = extended_address
            df["venueid"] = venueid
            df["location"] = location
            df["venuename"] = venuename
            df['postal_code'] = postal_code
            df["venuescore"] = venuescore
            df["venueslug"] = venueslug
            df["venuestate"] = state
            df["venuetimezone"] = timezone
            df['url'] = url
        else:
            df[key] = lst


    if count == 0:
        fulldf = df
    else:
        fulldf = fulldf.append(df, ignore_index = True)
    count = count + 1

In [6]:
# clean the data

# rename score as ticketscore to distinguish it from other scores
fulldf=fulldf.rename(columns = {'score':'ticket_score'})

# make sure you only have nba games
fulldf = fulldf.loc[fulldf['type'] == "nba"]
# get rid of the all-star game
fulldf = fulldf.loc[fulldf['title'] != "NBA All-Star Game"]

# drop unnecessary columns
fulldf = fulldf.drop("time_tbd", 1)
fulldf = fulldf.drop("taxonomies", 1)
fulldf = fulldf.drop("links", 1)
fulldf = fulldf.drop("url", 1)
fulldf = fulldf.drop("display_location", 1)
fulldf = fulldf.drop("venueslug", 1)
fulldf = fulldf.drop("venuetimezone", 1)
fulldf = fulldf.drop("date_tbd", 1)
fulldf = fulldf.drop("type", 1)
fulldf = fulldf.drop("location", 1)

# sources: http://stackoverflow.com/questions/17071871/select-rows-from-a-dataframe-based-on-values-in-a-column-in-pandas

In [7]:
fulldf.head()

,listing_count,avg_price,lowest_price_good_deal,lowest_price,highest_price,title,datetime_local,created_at,performers,address,city,country,extended_address,venueid,venuename,postal_code,venuescore,venuestate,announce_date,short_title,datetime_utc,ticket_score,datetime_tbd,visible_until_utc,id
0,352,107.5,27,27,489,Boston Celtics at San Antonio Spurs,2015-12-05T19:30:00,2015-08-13T00:00:00,"[{u'away_team': True, u'stats': {u'event_count...",1 AT&T Center,San Antonio,US,"San Antonio, TX 78219",185,AT&T Center,78219,0.76474,TX,2015-08-13T00:00:00,Celtics at Spurs,2015-12-06T01:30:00,0.74471,False,2015-12-06T05:30:00,2759759
1,1094,31.0,5,5,195,Boston Celtics at New Orleans Pelicans,2015-12-07T19:00:00,2015-08-13T00:00:00,"[{u'away_team': True, u'stats': {u'event_count...",1501 Girod Street,New Orleans,US,"New Orleans, LA 70113",184,Smoothie King Center,70113,0.75717,LA,2015-08-13T00:00:00,Celtics at Pelicans,2015-12-08T01:00:00,0.73357,False,2015-12-08T05:00:00,2759738
2,1794,72.0,23,23,606,Chicago Bulls at Boston Celtics,2015-12-09T19:00:00,2015-08-13T00:00:00,"[{u'home_team': True, u'stats': {u'event_count...",100 Legends Way,Boston,US,"Boston, MA 02114",1544,TD Garden,02114,0.85475,MA,2015-08-13T00:00:00,Bulls at Celtics,2015-12-10T00:00:00,0.79688,False,2015-12-10T04:00:00,2759699
3,767,368.5,129,129,3283,Golden State Warriors at Boston Celtics,2015-12-11T19:30:00,2015-08-13T00:00:00,"[{u'home_team': True, u'stats': {u'event_count...",100 Legends Way,Boston,US,"Boston, MA 02114",1544,TD Garden,02114,0.85475,MA,2015-08-13T00:00:00,Warriors at Celtics,2015-12-12T00:30:00,0.79955,False,2015-12-12T04:30:00,2760121
6,459,101.0,28,28,116676,Boston Celtics at Charlotte Hornets,2015-12-12T19:00:00,2015-08-13T00:00:00,"[{u'away_team': True, u'stats': {u'event_count...",333 East Trade Street,Charlotte,US,"Charlotte, NC 28202",112,Time Warner Cable Arena,28202,0.74988,NC,2015-08-13T00:00:00,Celtics at Hornets,2015-12-13T00:00:00,0.71937,False,2015-12-13T04:00:00,2759887


In [8]:
# put teams in separate columns

teams_playing = fulldf.title
away_teams = []
home_teams = []
for teams_pl in teams_playing:
    split = teams_pl.split(" at ")
    away_teams.append(split[0])
    home_teams.append(split[1])
    
fulldf["home_team"] = home_teams
fulldf["away_team"] = away_teams
fulldf = fulldf.drop("title", 1)

In [9]:
# unpack performers dicts

performers = fulldf.performers
home_teams = list(fulldf["home_team"])
away_teams = list(fulldf["away_team"])

hometeamscore = []
awayteamscore = []
count = 0
lens = []
for perf in performers:
    lens.append(len(perf))
    for p in perf:
        if p.keys()[0] == "home_team":
            hometeamscore.append(p["score"])
        elif p.keys()[0] == "away_team":
            awayteamscore.append(p["score"])
    count = count + 1

fulldf["hometeamscore"] = hometeamscore
fulldf["awayteamscore"] = awayteamscore

fulldf = fulldf.drop("performers", 1)
# sources: http://stackoverflow.com/questions/13411544/delete-column-from-pandas-dataframe

In [10]:
# calculate days away

import time
import datetime

datetimes = fulldf["datetime_utc"]
date = []
timeutc = []
daysaway = []
toddate = datetime.datetime.strptime(time.strftime("%Y-%m-%d"), "%Y-%m-%d").date()

for dattime in datetimes:
    futdate = datetime.datetime.strptime(dattime.split("T")[0],"%Y-%m-%d").date()
    timeaway = futdate - toddate
    daysaway.append(int(timeaway.days))
    date.append(dattime.split("T")[0])
    timeutc.append(dattime.split("T")[1])
fulldf["dateutc"] = date
fulldf["timeutc"] = timeutc
fulldf["daysaway"] = daysaway

# sources: http://www.cyberciti.biz/faq/howto-get-current-date-time-in-python/

In [11]:
# reset index
fulldf = fulldf.reset_index()

# save as a CSV
fulldf.to_csv("basketballdf" + str(toddate) + ".csv")

# sources: http://chrisalbon.com/python/pandas_saving_dataframe_as_csv.html

In [411]:
# A little EDA:
# plt.scatter(fulldf["daysaway"], fulldf["lowest_price"])

# print np.mean(fulldf["lowest_price"][fulldf["daysaway"] < 90])
# print np.mean(fulldf["lowest_price"][fulldf["daysaway"] > 90])
#     # the hole at about 80 days is the NBA All Star Game

In [412]:
# OUTLINE:
# Short time horizon:
    # logistic for probability that appreciates more than x%
    # regression with y = final price after x days
# In order to extend to further time horizons without data, can regress today's price
    # get ceof. on days away
    # forecast by changing days away

In [360]:
fulldf.head()['avg_price']

0    108
1     29
2     72
3    366
4    101
Name: avg_price, dtype: float64

Linear Regression
y=Predict today’s price
x=location,team info,sports stats,days away from the game,sports,day of the week,location holidays, the number of sports teams, 

Method: Linear regression

In [337]:
# fulldfComp=pd.read_csv('basketballdf2015-11-25.csv')

# string_date=fulldf.head()['dateutc'][0]
# string_date=datetime.datetime.strptime(string_date, "%Y-%m-%d")
# delta_date=string_date-datetime.datetime.strptime('2015-12-04', "%Y-%m-%d")
# # fulldfComp.head()
# delta_date.days
# # datetime.datetime.strptime(fulldf['dateutc'], "%Y-%m-%d")
# fulldf['deltaDay']=fulldf['dateutc'].str.replace(r'[-,]', '').astype('float')
# dataFa='2015-12-04'.replace('-', '')
# print dataFa
# fulldf['deltaDay']=fulldf['deltaDay']-float(dataFa)
# fulldf.head()
fulldf

In [12]:
import re
web='https://en.wikipedia.org/wiki/List_of_American_and_Canadian_cities_by_number_of_major_professional_sports_franchises#cite_note-CFL_teams-8'
team1 = requests.get(web)
capTable = BeautifulSoup(team1.text)
resultTable= capTable.find_all("table",{'class':"wikitable sortable"})[0]

resultTable = resultTable.find_all('tr')
paragraphs = []
for num in range(len(resultTable)):
    if num==0:
        elem=resultTable[num].find_all('th')
    else:
        elem=resultTable[num].find_all('td')
    result=[]
    for elem1 in elem:
        result.append(elem1)
    paragraphs.append(result)

CLEAN = re.compile('\s+')
newpar=[]
for elem in paragraphs:
    elem = [CLEAN.sub(' ', a.text).strip() for a in elem]
    newpar.append(elem)

numCity = pd.DataFrame(newpar)
numCity.columns = numCity.iloc[0]
numCity=numCity.drop(numCity.index[0])

columnsTeam=[numCity.columns[0],numCity.columns[3],numCity.columns[4],numCity.columns[5],numCity.columns[6]]

numCity[numCity.columns[6]] = numCity[numCity.columns[6]].str[:1]
numCity[numCity.columns[7]] = numCity[numCity.columns[7]].str[:1]
numCity[numCity.columns[8]] = numCity[numCity.columns[8]].str[:1]
numCity[numCity.columns[9]] = numCity[numCity.columns[9]].str[:1]


columnsTeam=[numCity.columns[0],numCity.columns[3],numCity.columns[4],numCity.columns[5],numCity.columns[6],numCity.columns[7],numCity.columns[8],numCity.columns[9]]
dfCity = pd.DataFrame(numCity,index=numCity.index, columns=columnsTeam)
dfCity.columns = ['City','Population','B4','B6','NFL','MLB','NBA','NHL']
dfCity
dfCity['City'][13]='Minneapolis'
dfCity
dfCity=dfCity.drop(dfCity.index[len(dfCity)-1])

# dfCity[dfCity.columns[1]] = dfCity[dfCity.columns[1]].astype(float)
dfCity[dfCity.columns[2]] = dfCity[dfCity.columns[2]].astype(float)
dfCity[dfCity.columns[3]] = dfCity[dfCity.columns[3]].astype(float)
dfCity[dfCity.columns[4]] = dfCity[dfCity.columns[4]].astype(float)
dfCity[dfCity.columns[5]] = dfCity[dfCity.columns[5]].astype(float)
dfCity[dfCity.columns[6]] = dfCity[dfCity.columns[6]].astype(float)
dfCity[dfCity.columns[7]] = dfCity[dfCity.columns[7]].astype(float)
# df[['two', 'three']] = df[['two', 'three']].astype(float)
# str(dfCity['Population'][1]).replace
dfCity['Population']=dfCity['Population'].str.replace(',', '').astype('float')
dfCity

,City,Population,B4,B6,NFL,MLB,NBA,NHL
1,New York City,20630000,9,11,2,2,2,3
2,Los Angeles,15058000,6,7,0,2,2,2
3,San Francisco Bay Area,5929000,6,7,2,2,1,1
4,Chicago,9156000,5,6,1,2,1,1
5,Dallas-Fort Worth,6174000,4,5,1,1,1,1
6,Philadelphia,5570000,4,5,1,1,1,1
7,"Washington, D.C.",4889000,4,5,1,1,1,1
8,Boston,4478000,4,5,1,1,1,1
9,Denver,2559000,4,5,1,1,1,1
10,Miami,5764000,4,4,1,1,1,1


In [22]:
r1 = requests.get("http://espn.go.com/nba/standings/_/group/league")
text1 = r1.text
bball_data=pd.read_html(text1)
NBA=bball_data[0].copy()
#should I add an indicator for the division? (i.e. West vs. East)
NBA=NBA.rename(columns = {'Unnamed: 0':'Teams'})
#should I try to get rid of the extra letter in the Team Names?
# NBA['Teams']=NBA['Teams'][:6]
# fulldf['modCity']=fulldf['city']

NBA[NBA.columns[0]] = NBA[NBA.columns[0]].str[:-3]
# NBA
NBA.loc[NBA['Teams']=='Golden State Warrior','Teams']='Golden State Warriors'
NBA.loc[NBA['Teams']=='San Antonio Spur','Teams']='San Antonio Spurs'
# fulldf.loc[fulldf['modCity']=='Minneapolis','modCity']='Minneapolis–Saint Paul'
NBA.loc[NBA['Teams']=='New York Knick','Teams']='New York Knicks'
NBA.loc[NBA['Teams']=='Utah JazzU','Teams']='Utah Jazz'
NBA.loc[NBA['Teams']=='New Orleans Pelican','Teams']='New Orleans Pelicans'
NBA=NBA.rename(columns = {'PCT':'WPCT'})
awayNBA=NBA.rename(columns = {'Teams':'away_team'})
awayNBA=awayNBA.rename(columns = {'WPCT':'awayWPCT'})
homeNBA=NBA.rename(columns = {'Teams':'home_team'})
homeNBA=homeNBA.rename(columns = {'WPCT':'homeWPCT'})

In [37]:
rankNBA=pd.read_csv('Sports_Data/NBA2015-12-04.csv')
# rankNBA=rankNBA.rename(columns = {'Unnamed: 0':'Teams'})
del rankNBA['Unnamed: 0']
rankNBA[rankNBA.columns[0]] = rankNBA[rankNBA.columns[0]].str[:-3]
# rankNBA
rankNBA.loc[rankNBA['Teams']=='Golden State Warrior','Teams']='Golden State Warriors'
rankNBA.loc[rankNBA['Teams']=='San Antonio Spur','Teams']='San Antonio Spurs'
# fulldf.loc[fulldf['modCity']=='Minneapolis','modCity']='Minneapolis–Saint Paul'
rankNBA.loc[rankNBA['Teams']=='New York Knick','Teams']='New York Knicks'
rankNBA.loc[rankNBA['Teams']=='Utah JazzU','Teams']='Utah Jazz'
rankNBA.loc[rankNBA['Teams']=='New Orleans Pelican','Teams']='New Orleans Pelicans'
rankNBA=rankNBA.rename(columns = {'PCT':'WPCT'})
awrankNBA=rankNBA.rename(columns = {'Teams':'away_team'})
awrankNBA=awrankNBA.rename(columns = {'WPCT':'awayWPCT'})
hmrankNBA=rankNBA.rename(columns = {'Teams':'home_team'})
hmrankNBA=hmrankNBA.rename(columns = {'WPCT':'homeWPCT'})
hmrankNBA.head()
awrankNBA.head()


,away_team,W,L,awayWPCT,GB,HOME,ROAD,DIV,CONF,PPG,OPP PPG,DIFF,STRK,L10
0,Golden State Warriors,20,0,1.000,-,10-0,10-0,6-0,15-0,115.3,99.9,15.4,W20,10-0
1,San Antonio Spurs,16,4,0.800,4,10-0,6-4,3-1,9-2,98.8,88.5,10.3,W2,8-2
2,Chicago Bulls,11,5,0.688,7,7-1,4-4,2-2,6-3,98.9,97.8,1.1,W2,7-3
3,Cleveland Cavaliers,13,6,0.684,6.5,9-1,4-5,2-3,11-5,102.4,97.5,4.9,L2,5-5
4,Indiana Pacers,12,6,0.667,7,6-2,6-4,3-2,9-3,102.4,96.8,5.6,L1,8-2


In [41]:
# r4 = requests.get("http://espn.go.com/nba/attendance")
# text4 = r4.text
# NBAattn_data=pd.read_html(text4)
# NBA_attendance=NBAattn_data[0].copy()
# NBA_attendance.columns = NBA_attendance.iloc[1]
# NBA_attendance=NBA_attendance.reindex(NBA_attendance.index.drop(0))
# NBA_attendance=NBA_attendance[NBA_attendance.RK != 'RK'].reset_index(drop=True)
# NBA_attendance.head(5)
# NBA_attendance['TEAM']

NBA_attendance=pd.read_csv('Attendance/NBA2015-11-29.csv')
del NBA_attendance['Unnamed: 0']
NBA_attendance.loc[NBA_attendance['TEAM']=='Bulls','TEAM']='Chicago Bulls'
NBA_attendance.loc[NBA_attendance['TEAM']=='Cavaliers','TEAM']='Cleveland Cavaliers'
NBA_attendance.loc[NBA_attendance['TEAM']=='Mavericks','TEAM']='Dallas Mavericks'
NBA_attendance.loc[NBA_attendance['TEAM']=='Raptors','TEAM']='Toronto Raptors'
NBA_attendance.loc[NBA_attendance['TEAM']=='NY Knicks','TEAM']='New York Knicks'
NBA_attendance.loc[NBA_attendance['TEAM']=='Heat','TEAM']='Miami Heat'

NBA_attendance.loc[NBA_attendance['TEAM']=='Warriors','TEAM']='Golden State Warriors'
NBA_attendance.loc[NBA_attendance['TEAM']=='Jazz','TEAM']='Utah Jazz'
NBA_attendance.loc[NBA_attendance['TEAM']=='Trail Blazers','TEAM']='Portland Trail Blazers'
NBA_attendance.loc[NBA_attendance['TEAM']=='Raptors','TEAM']='Toronto Raptors'
NBA_attendance.loc[NBA_attendance['TEAM']=='Clippers','TEAM']='Los Angeles Clippers'
NBA_attendance.loc[NBA_attendance['TEAM']=='Lakers','TEAM']='Los Angeles Lakers'

NBA_attendance.loc[NBA_attendance['TEAM']=='Spurs','TEAM']='San Antonio Spurs'
NBA_attendance.loc[NBA_attendance['TEAM']=='Thunder','TEAM']='Oklahoma City Thunder'
NBA_attendance.loc[NBA_attendance['TEAM']=='Wizards','TEAM']='Washington Wizards'
NBA_attendance.loc[NBA_attendance['TEAM']=='Celtics','TEAM']='Boston Celtics'
NBA_attendance.loc[NBA_attendance['TEAM']=='Rockets','TEAM']='Houston Rockets'
NBA_attendance.loc[NBA_attendance['TEAM']=='Kings','TEAM']='Sacramento Kings'

NBA_attendance.loc[NBA_attendance['TEAM']=='Magic','TEAM']='Orlando Magic'
NBA_attendance.loc[NBA_attendance['TEAM']=='Suns','TEAM']='Phenoix Suns'
NBA_attendance.loc[NBA_attendance['TEAM']=='Grizzlies','TEAM']='Memphis Grizzlies'
NBA_attendance.loc[NBA_attendance['TEAM']=='Pelicans','TEAM']='New Orleans Pelicans'
NBA_attendance.loc[NBA_attendance['TEAM']=='Hornets','TEAM']='Charlotte Hornets'
NBA_attendance.loc[NBA_attendance['TEAM']=='Pacers','TEAM']='Indiana Pacers'

NBA_attendance.loc[NBA_attendance['TEAM']=='Hawks','TEAM']='Atlanta Hawks'
NBA_attendance.loc[NBA_attendance['TEAM']=='Pistons','TEAM']='Detroit Pistons'
NBA_attendance.loc[NBA_attendance['TEAM']=='76ers','TEAM']='Philadelphia 76ers'
NBA_attendance.loc[NBA_attendance['TEAM']=='Nets','TEAM']='Brooklyn Nets'
NBA_attendance.loc[NBA_attendance['TEAM']=='Bucks','TEAM']='Milwaukee Bucks'
NBA_attendance.loc[NBA_attendance['TEAM']=='Timberwolves','TEAM']='Minnesota Timberwolves'
NBA_attendance.loc[NBA_attendance['TEAM']=='Nuggets','TEAM']='Denver Nuggets'
# NBA_attendance.columns = [ 'RK', 'TEAM', 'GMS','TOTAL','avgAud','PCT','GMS','AVG','PCT','GMS','AVG','PCT']
# # home_team
NBA_attendance['AVG']=NBA_attendance['AVG'].astype('float')
NBA_attendance=NBA_attendance.rename(columns = {'TEAM':'home_team'})
NBA_attendance=NBA_attendance.rename(columns = {'AVG':'avgAud'})
NBA_attendance

,RK,home_team,GMS,TOTAL,avgAud,PCT,GMS.1,AVG.1,PCT.1,GMS.2,AVG.2,PCT.2
0,1,Chicago Bulls,6,130800,21800,104.2,8,17164,87.7,14,19150,95.1
1,2,Cleveland Cavaliers,9,185058,20562,100.0,8,19296,97.2,17,19966,98.7
2,3,Dallas Mavericks,7,140555,20079,104.6,10,17444,94.0,17,18529,98.4
3,4,Toronto Raptors,5,99340,19868,100.3,12,17858,93.2,17,18449,95.3
4,5,New York Knicks,8,158496,19812,100.0,9,19014,98.8,17,19389,99.4
5,6,Miami Heat,10,196599,19659,100.3,5,16840,84.2,15,18720,94.9
6,7,Golden State Warriors,10,195960,19596,100.0,8,17923,97.8,18,18852,99.0
7,8,Utah Jazz,5,97930,19586,98.4,10,17026,87.0,15,17879,90.8
8,9,Portland Trail Blazers,8,154283,19285,92.7,9,16975,90.0,17,18062,91.4
9,10,Los Angeles Clippers,10,191629,19162,100.5,6,17785,93.2,16,18646,97.8


In [16]:
t=set(fulldf['city'].values)

s=set(dfCity['City'].values)

# print t.difference(s)

# df.loc[df['First Season'] > 1990, 'First Season'] = 1
fulldf['modCity']=fulldf['city']
fulldf.loc[fulldf['modCity']=='Washington','modCity']='Washington, D.C.'
fulldf.loc[fulldf['modCity']=='Dallas','modCity']='Dallas-Fort Worth'
# fulldf.loc[fulldf['modCity']=='Minneapolis','modCity']='Minneapolis–Saint Paul'
fulldf.loc[fulldf['modCity']=='New York','modCity']='New York City'
fulldf.loc[fulldf['modCity']=='Brooklyn','modCity']='New York City'
fulldf.loc[fulldf['modCity']=='Oakland','modCity']='San Francisco Bay Area'
fulldf['City']=fulldf['modCity']
# Minneapolis–Saint Paul
# df.loc[df['First Season'] > 1990, 'First Season'] = 1
# fulldf = fulldf[fulldf.city in t]
fulldf.head()
newt=set(fulldf['modCity'].values)
# print newt.difference(s)
fulldf.head()

fulldf['home_team'] = fulldf['home_team'].str[:]
fulldf['away_team'] = fulldf['away_team'].str[:]
# NBA=NBA.rename(columns = {'home_team':'Teams'})
fulldf=pd.merge(fulldf, homeNBA, how='left', on='home_team')
fulldf=pd.merge(fulldf, awayNBA, how='left', on='away_team')
fulldf=pd.merge(fulldf, NBA_attendance, how='left', on='home_team')
fulldf=pd.merge(fulldf, dfCity, how='left', on='City')
# fulldf=pd.concat([fulldf, dfCity], axis=1, join_axes=[dfCity.index], join='inner')
fulldf.head()

,index,listing_count,avg_price,lowest_price_good_deal,lowest_price,highest_price,datetime_local,created_at,address,city,country,extended_address,venueid,venuename,postal_code,venuescore,venuestate,announce_date,short_title,datetime_utc,ticket_score,datetime_tbd,visible_until_utc,id,home_team,away_team,hometeamscore,awayteamscore,dateutc,timeutc,daysaway,modCity,City,W_x,L_x,homeWPCT,GB_x,HOME_x,ROAD_x,DIV_x,CONF_x,PPG_x,OPP PPG_x,DIFF_x,STRK_x,L10_x,W_y,L_y,awayWPCT,GB_y,HOME_y,ROAD_y,DIV_y,CONF_y,PPG_y,OPP PPG_y,DIFF_y,STRK_y,L10_y,RK,GMS,TOTAL,avgAud,PCT,GMS,AVG,PCT,GMS,AVG,PCT,Population,B4,B6,NFL,MLB,NBA,NHL
0,0,352,107.5,27,27,489,2015-12-05T19:30:00,2015-08-13T00:00:00,1 AT&T Center,San Antonio,US,"San Antonio, TX 78219",185,AT&T Center,78219,0.76474,TX,2015-08-13T00:00:00,Celtics at Spurs,2015-12-06T01:30:00,0.74471,False,2015-12-06T05:30:00,2759759,San Antonio Spurs,Boston Celtics,0.70475,0.74908,2015-12-06,01:30:00,1,San Antonio,San Antonio,16,4,0.800,4,10-0,6-4,3-1,9-2,98.8,88.5,10.3,W2,8-2,11,8,0.579,8.5,6-4,5-4,3-2,8-6,102.6,97.5,5.2,W2,6-4,12,10,184180,18418,99.1,10,18264,95.9,20,18341,97.5,1976000,1,1,0,0,1,0
1,1,1094,31.0,5,5,195,2015-12-07T19:00:00,2015-08-13T00:00:00,1501 Girod Street,New Orleans,US,"New Orleans, LA 70113",184,Smoothie King Center,70113,0.75717,LA,2015-08-13T00:00:00,Celtics at Pelicans,2015-12-08T01:00:00,0.73357,False,2015-12-08T05:00:00,2759738,New Orleans Pelicans,Boston Celtics,0.64723,0.74908,2015-12-08,01:00:00,3,New Orleans,New Orleans,5,15,0.250,15,4-5,1-10,2-3,4-10,102.3,109.0,-6.8,W1,4-6,11,8,0.579,8.5,6-4,5-4,3-2,8-6,102.6,97.5,5.2,W2,6-4,21,9,153966,17107,99.5,11,18691,97.2,20,17978,98.2,922000,2,2,1,0,1,0
2,2,1794,72.0,23,23,606,2015-12-09T19:00:00,2015-08-13T00:00:00,100 Legends Way,Boston,US,"Boston, MA 02114",1544,TD Garden,02114,0.85475,MA,2015-08-13T00:00:00,Bulls at Celtics,2015-12-10T00:00:00,0.79688,False,2015-12-10T04:00:00,2759699,Boston Celtics,Chicago Bulls,0.74908,0.76150,2015-12-10,00:00:00,5,Boston,Boston,11,8,0.579,8.5,6-4,5-4,3-2,8-6,102.6,97.5,5.2,W2,6-4,11,5,0.688,7,7-1,4-4,2-2,6-3,98.9,97.8,1.1,W2,7-3,15,10,178459,17845,95.8,9,16595,90.1,19,17253,93.1,4478000,4,5,1,1,1,1
3,3,767,368.5,129,129,3283,2015-12-11T19:30:00,2015-08-13T00:00:00,100 Legends Way,Boston,US,"Boston, MA 02114",1544,TD Garden,02114,0.85475,MA,2015-08-13T00:00:00,Warriors at Celtics,2015-12-12T00:30:00,0.79955,False,2015-12-12T04:30:00,2760121,Boston Celtics,Golden State Warriors,0.74908,0.75952,2015-12-12,00:30:00,7,Boston,Boston,11,8,0.579,8.5,6-4,5-4,3-2,8-6,102.6,97.5,5.2,W2,6-4,20,0,1.000,-,10-0,10-0,6-0,15-0,115.3,99.9,15.4,W20,10-0,15,10,178459,17845,95.8,9,16595,90.1,19,17253,93.1,4478000,4,5,1,1,1,1
4,6,459,101.0,28,28,116676,2015-12-12T19:00:00,2015-08-13T00:00:00,333 East Trade Street,Charlotte,US,"Charlotte, NC 28202",112,Time Warner Cable Arena,28202,0.74988,NC,2015-08-13T00:00:00,Celtics at Hornets,2015-12-13T00:00:00,0.71937,False,2015-12-13T04:00:00,2759887,Charlotte Hornets,Boston Celtics,0.66337,0.74908,2015-12-13,00:00:00,8,Charlotte,Charlotte,10,8,0.556,9,8-3,2-5,1-3,6-6,102.3,99.7,2.6,L1,6-4,11,8,0.579,8.5,6-4,5-4,3-2,8-6,102.6,97.5,5.2,W2,6-4,22,11,181839,16530,86.7,7,18726,96.2,18,17384,90.4,1535000,2,2,1,0,1,0


In [20]:
# fulldfComp
import numpy as np
from sklearn import datasets, linear_model

import statsmodels.api as sm

X = fulldf[['venuescore', 'ticket_score','daysaway','B4','Population','homeWPCT','awayWPCT','avgAud']]
y = fulldf['lowest_price']
# lowest_price~daysaway+home_team+away_team
## fit a OLS model with intercept on TV and Radio
X = sm.add_constant(X)
est = sm.OLS(y, X,missing = 'drop').fit()
est.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           lowest_price   R-squared:                       0.229
Model:                            OLS   Adj. R-squared:                  0.225
Method:                 Least Squares   F-statistic:                     64.73
Date:                Sat, 05 Dec 2015   Prob (F-statistic):           4.45e-93
Time:                        10:08:28   Log-Likelihood:                -8733.6
No. Observations:                1756   AIC:                         1.749e+04
Df Residuals:                    1747   BIC:                         1.753e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
const          -85.4347     20.614     -4.145      0.000      -125.865   -45.005
venuescore    -242.3490     38.913     -6.228      0.000      -318.670  -166.028
ticket_score   265.1418     34.381      7.712      0.000       197.710   332.573
daysaway        -0.0409      0.022     -1.897      0.058        -0.083     0.001
B4               7.4624      1.073      6.958      0.000         5.359     9.566
Population   -6.112e-07    4.2e-07     -1.454      0.146     -1.44e-06  2.13e-07
homeWPCT        28.7653      5.546      5.187      0.000        17.889    39.642
awayWPCT        23.8744      4.751      5.026      0.000        14.557    33.192
avgAud           0.0038      0.001      7.164      0.000         0.003     0.005
==============================================================================
Omnibus:                     2133.554   Durbin-Watson:                   1.544
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           472667.298
Skew:                           6.095   Prob(JB):                         0.00
Kurtosis:                      82.445   Cond. No.                     4.47e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.47e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""